<h1 style="font-family: Arial, Helvetica, sans-serif; text-align:center;">Lightning Dashboard <h1>

<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">
<h1 style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    **Showing how COG (Cloud Optimized Geotiff Images) are visualized with mapbox using TRMM_LIS_FULL as a sample example.**
</h1>
<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold">
    TRMM_LIS_FULL netcdf4 file is grabbed from the EDS and is converted to COG using the rasterio library. COGs are then added to the s3 bucket and the tiling server (terracotta) is configured then deployed to the AWS.
</span>

In [91]:
#base url of the deployed terracotta server
base_url = "https://wug8w3fg42.execute-api.us-west-2.amazonaws.com/development/singleband/"

#url for the trmm lis full dataset
trmm_lis_full = "VHRFC/201301/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.325,23.426]"

#sample url for other datasets (proper date/time need to be added to each url to get proper COG response from the server)
trmm_lis_seasonal = "VHRSC/{date}/LIS/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0.00010455249866936,0.06766455620527]"
otd_monthly = "HRMC/{date}/OTD/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0.2]"
isslis = "ISS_LIS/{date}/{time}/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,0]"
nalma = "NALMA_{time}/{date}/{band}/{z}/{x}/{y}.png?colormap=terrain&stretch_range=[0,1]"

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Optionally installing the required libraries depending on whether the system already has it
</span>

In [92]:
import importlib

# List of packages to install
packages_to_install = [
    'plotly',
    'rasterio',
    'pandas',
    'requests',
]

for package_name in packages_to_install:
    if importlib.util.find_spec(package_name) is None:
        !pip install {package_name}
    else:
        print(f"{package_name} is already installed.")

plotly is already installed.
rasterio is already installed.
pandas is already installed.
requests is already installed.


<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Importing Libraries used for Visualization
</span>

In [93]:
import plotly.io as pio
import plotly.express as px
import rasterio
from rasterio.plot import show
import pandas as pd
import requests

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Function Below visualizes COG on mapbox. COG image url needs to be passed
</span>

In [94]:
#this function visualized any given COG inside of a mapbox view
def visualize_COG(layer_url, zoom=1, center={"lat": 33.543682, "lon": -86.779633}):
    pio.renderers.default = 'iframe'
    styles_list = [
        "carto-darkmatter",
        "carto-positron",
        "open-street-map",
        "stamen-terrain",
        "stamen-toner",
        "stamen-watercolor",
        "white-bg"
    ]
    style = styles_list[1]
    # seed value, for mapbox to load in colab
    df = pd.DataFrame([[1001, 5.3],[1001, 5.3]])
    df.columns = ["flips", "unemp"]
    fig = px.choropleth_mapbox(
                                df, 
                                color='unemp',
                                color_continuous_scale="Viridis",
                                range_color=(0, 12),
                                mapbox_style=style,
                                center=center,
                                opacity=1,
                                zoom=zoom 
                              )
    fig.update_layout(
        mapbox_layers=[
            {
                "sourcetype": "raster",
                "source": [layer_url]
            }
          ])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(mapbox_style=style)
    fig.layout.mapbox.zoom = zoom
    fig.show()

<span style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Calling the visualize function (TRMM LIS FULL is visualized on the Mapbox)
</span>

In [95]:
visualize_COG(base_url+trmm_lis_full)

<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">
<h1 style="font-family: Arial, Helvetica, sans-serif; font-size: 16px; font-weight:bold; text-align:center">
    Package below uses the same concept as above to visualize all the current available datasets on the Lightning Dashboard.
</h1>
<hr style="background-color: black; height: 2px; border: none; margin: 5px 0; font-weight: bold;">

In [96]:
from src.Lightning2DVisualization.inputHandlers import drop_down

In [97]:
drop_down()

Select a dataset from the options below: 

Dropdown(options=('TRMM_LIS_FULL', 'TRMM_LIS_SEASONAL', 'TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAIL…

Output()